In [1]:
!pip install -r ../requirements.txt

In [ ]:
import os, torch, time, gc, itertools
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from data_module import parallel_load, layernorm
from model_module import Classifier, calculate_metrics, test_model

In [ ]:
data_path = '../data/exam_data'
model_path = '../model'
result_path = '../result'

os.makedirs(model_path, exist_ok=True)
os.makedirs(os.path.join(result_path, 'predict'), exist_ok=True)

embed_ver = ['esm3', 'esm2', 'bert', 't5']
embed_types = ['allmean', 'aamean', 'bos', 'eos', 'first', 'center', 'last']
set_ver = 'ts'
batch_size = 256

In [ ]:
info_col = ['file_id', 'organism', 'locus_tag', 'ess']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_workers = min(os.cpu_count(), 16)

In [ ]:
time_total = time.time()
df_eval = pd.DataFrame()
for emb_ver in embed_ver:
    print(f">>> {emb_ver.upper()} <<<")
    time_emb = time.time()
    
    # load training datasets of the embedding version
    print("> Dataset loading...")
    time_start = time.time()
    df_info = pd.read_hdf(os.path.join(data_path, f'emb-{emb_ver}_{set_ver}.h5'), key='info')
    dfs = parallel_load(embed_types, data_path, 'emb', emb_ver, set_ver)
    gc.collect()
    print(f"--- Dataset loading complete: {time.time() - time_start:.1f} sec ---")
    
    for r in range(1, len(embed_types[:-2]) + 1):
        emb_types = embed_types[:-2] if r > 1 else embed_types

        for df_keys in itertools.combinations(emb_types, r):
            comb_ver = '_'.join([df_key for df_key in df_keys])
            if 't5' in emb_ver.lower() and 'bos' in comb_ver.lower():
                continue
            
            time_start = time.time()
            for integ_ver in ['indiv', 'conf', 'sum', 'cat']:
                model_ver = f'{emb_ver}-{integ_ver}-{comb_ver}'

                if r == 1 and integ_ver != 'indiv':
                    continue
                
                elif r > 1 and integ_ver == 'indiv':
                    continue
                
                #### Confidence ensemble ####
                elif r > 1 and integ_ver == 'conf':
                    print(f"> '{model_ver}' prediction data setting...")
                    # load prediction data
                    pred_dfs = {}
                    for emb_type in df_keys:
                        pred_dfs[emb_type] = pd.read_csv(os.path.join(result_path, f'predict/{emb_ver}-indiv-{emb_type}.csv'))
                                        
                    # merge data
                    preds = pred_dfs[df_keys[0]].copy()
                    for df_key in df_keys[1:]:
                        preds = pd.merge(preds, pred_dfs[df_key], on=info_col, suffixes=("", f"_{df_key}"))
                    print(f"- '{model_ver}' prediction data setting complete: {time.time() - time_start:.1f} sec")

                    # calculate mean of confidences
                    col_num = [col for col in preds.columns if col not in info_col]
                    preds['conf_mean'] = preds[col_num].mean(axis=1)

                    # evaluate the confidence ensemble
                    print(f"> '{model_ver}' test...")
                    time_start = time.time()
                    md_eval = calculate_metrics(
                        embed_ver=emb_ver,
                        integ_ver=integ_ver,
                        comb_ver=comb_ver,
                        y_true=preds['ess'].to_numpy(),
                        y_prob=preds['conf_mean'].to_numpy(),
                        y_pred=np.round(preds['conf_mean'].to_numpy())
                    )

                    # gather the result
                    df_eval = pd.concat([df_eval, md_eval])
                    print(f"- '{model_ver}' test compelete: {time.time() - time_start:.1f} sec")

                    # save the predictions
                    print(f"- '{model_ver}' prediction save...")
                    time_start = time.time()
                    preds.to_csv(
                        os.path.join(result_path, f'predict/{model_ver}.csv'),
                        index=False
                    )
                    display(preds)
                    print(f"--- '{model_ver}' prediction save complete: {time.time() - time_start:.1f} sec ---\n")
                    
                    # release memory
                    del pred_dfs, preds
                    gc.collect()
                    continue
                
                #### Data preprocessing ####
                else:
                    print(f"> '{model_ver}' data-loader setting...")                    
                    embeds = dfs[df_keys[0]].to_numpy()
                    if integ_ver == 'sum':
                        # sum normalized dataset
                        embeds = layernorm(embeds)
                        for df_key in df_keys[1:]:
                            embeds += layernorm(dfs[df_key].to_numpy())
                    elif integ_ver == 'cat':
                        # concatenate dataset
                        for df_key in df_keys[1:]:
                            embeds = np.concatenate([embeds, dfs[df_key].to_numpy()], axis=1)

                # get label data
                labels = df_info['ess'].to_numpy('float')
                print(f"- '{model_ver}' input & output shape: {embeds.shape}, {labels.shape}")

                # set dataloader
                embeds = torch.FloatTensor(embeds)
                labels = torch.FloatTensor(labels)
                test_dataset = TensorDataset(embeds, labels)
                test_loader = DataLoader(
                    test_dataset, batch_size=batch_size,
                    pin_memory=True, num_workers=num_workers, prefetch_factor=8
                )            
                print(f"- '{model_ver}' data-loader setting complete: {time.time() - time_start:.1f} sec")


                #### Model evaluation ####
                # set model name
                print(f"> '{model_ver}' model test...")
                
                # generate modeling instances
                model = Classifier(input_size=embeds.shape[-1]).to(device)
                model.load_state_dict(
                    torch.load(
                        os.path.join(model_path, f'{model_ver}.pt'),
                        map_location=device
                    )
                )
                model.eval()
                
                # test the model
                time_start = time.time()            
                md_eval, md_prob = test_model(
                    model=model,
                    test_loader=test_loader,
                    embed_ver=emb_ver,
                    integ_ver=integ_ver,
                    comb_ver=comb_ver,
                    info_df=df_info
                )
                
                # gather the result
                df_eval = pd.concat([df_eval, md_eval])
                print(f"- '{model_ver}' test compelete: {time.time() - time_start:.1f} sec")
                
                # save the predictions
                print(f"> '{model_ver}' prediction save...")
                time_start = time.time()
                md_prob.to_csv(
                    os.path.join(result_path, f'predict/{model_ver}.csv'),
                    index=False
                )
                display(md_prob)
                print(f"--- '{model_ver}' prediction save complete: {time.time() - time_start:.1f} sec ---\n")
                
                # release memory
                del model, embeds, labels, md_prob
                torch.cuda.empty_cache()
                gc.collect()
    
    print(f"=== '{emb_ver}' classifiers test complete: {time.time() - time_emb:.1f} sec ===\n")
# save the all model evaluation
df_eval.to_csv(os.path.join(result_path, f'eval-classifiers.csv'), index=False)
print(f"=== All test complete: {time.time() - time_total:.1f} sec ===")

>>> ESM3 <<<
> Dataset loading...
--- Dataset loading complete: 3.0 sec ---
> 'esm3-indiv-allmean' data-loader setting...
Input & output shape: (4313, 1152), (4313,)
--- Data-loader setting complete: 0.0 sec ---
> 'esm3-indiv-allmean' model test...
--- Test compelete: 2.6 sec ---
> Model prediction save...


,file_id,organism,locus_tag,ess,conf
0,C018,Escherichia coli K-12 BW25113,BW25113_0001,0,0.001919
1,C018,Escherichia coli K-12 BW25113,BW25113_0002,0,0.198053
2,C018,Escherichia coli K-12 BW25113,BW25113_0003,0,0.009606
3,C018,Escherichia coli K-12 BW25113,BW25113_0004,0,0.351769
4,C018,Escherichia coli K-12 BW25113,BW25113_0005,0,0.258920
...,...,...,...,...,...
4308,C018,Escherichia coli K-12 BW25113,BW25113_4702,0,0.229250
4309,C018,Escherichia coli K-12 BW25113,BW25113_4703,0,0.191968
4310,C018,Escherichia coli K-12 BW25113,BW25113_4705,0,0.011843
4311,C018,Escherichia coli K-12 BW25113,BW25113_4706,0,0.092694


--- Prediction save complete: 0.0 sec ---

> 'esm3-indiv-aamean' data-loader setting...
Input & output shape: (4313, 1152), (4313,)
--- Data-loader setting complete: 0.0 sec ---
> 'esm3-indiv-aamean' model test...
--- Test compelete: 0.5 sec ---
> Model prediction save...


,file_id,organism,locus_tag,ess,conf
0,C018,Escherichia coli K-12 BW25113,BW25113_0001,0,0.002807
1,C018,Escherichia coli K-12 BW25113,BW25113_0002,0,0.057684
2,C018,Escherichia coli K-12 BW25113,BW25113_0003,0,0.013577
3,C018,Escherichia coli K-12 BW25113,BW25113_0004,0,0.515560
4,C018,Escherichia coli K-12 BW25113,BW25113_0005,0,0.005336
...,...,...,...,...,...
4308,C018,Escherichia coli K-12 BW25113,BW25113_4702,0,0.117973
4309,C018,Escherichia coli K-12 BW25113,BW25113_4703,0,0.210178
4310,C018,Escherichia coli K-12 BW25113,BW25113_4705,0,0.007678
4311,C018,Escherichia coli K-12 BW25113,BW25113_4706,0,0.152543


--- Prediction save complete: 0.0 sec ---

> 'esm3-indiv-bos' data-loader setting...
Input & output shape: (4313, 1152), (4313,)
--- Data-loader setting complete: 0.0 sec ---
> 'esm3-indiv-bos' model test...
--- Test compelete: 0.5 sec ---
> Model prediction save...


...

--- Prediction save complete: 0.0 sec ---

> 't5-sum-allmean_aamean_eos_first' data-loader setting...
Input & output shape: (4313, 1024), (4313,)
--- Data-loader setting complete: 0.1 sec ---
> 't5-sum-allmean_aamean_eos_first' model test...
--- Test compelete: 0.5 sec ---
> Model prediction save...


,file_id,organism,locus_tag,ess,conf
0,C018,Escherichia coli K-12 BW25113,BW25113_0001,0,0.063164
1,C018,Escherichia coli K-12 BW25113,BW25113_0002,0,0.094376
2,C018,Escherichia coli K-12 BW25113,BW25113_0003,0,0.109721
3,C018,Escherichia coli K-12 BW25113,BW25113_0004,0,0.236014
4,C018,Escherichia coli K-12 BW25113,BW25113_0005,0,0.184351
...,...,...,...,...,...
4308,C018,Escherichia coli K-12 BW25113,BW25113_4702,0,0.142372
4309,C018,Escherichia coli K-12 BW25113,BW25113_4703,0,0.456425
4310,C018,Escherichia coli K-12 BW25113,BW25113_4705,0,0.011750
4311,C018,Escherichia coli K-12 BW25113,BW25113_4706,0,0.339202


--- Prediction save complete: 0.0 sec ---

> 't5-cat-allmean_aamean_eos_first' data-loader setting...
Input & output shape: (4313, 4096), (4313,)
--- Data-loader setting complete: 0.2 sec ---
> 't5-cat-allmean_aamean_eos_first' model test...
--- Test compelete: 0.7 sec ---
> Model prediction save...


,file_id,organism,locus_tag,ess,conf
0,C018,Escherichia coli K-12 BW25113,BW25113_0001,0,0.129349
1,C018,Escherichia coli K-12 BW25113,BW25113_0002,0,0.500768
2,C018,Escherichia coli K-12 BW25113,BW25113_0003,0,0.156536
3,C018,Escherichia coli K-12 BW25113,BW25113_0004,0,0.445817
4,C018,Escherichia coli K-12 BW25113,BW25113_0005,0,0.109656
...,...,...,...,...,...
4308,C018,Escherichia coli K-12 BW25113,BW25113_4702,0,0.122317
4309,C018,Escherichia coli K-12 BW25113,BW25113_4703,0,0.367636
4310,C018,Escherichia coli K-12 BW25113,BW25113_4705,0,0.440188
4311,C018,Escherichia coli K-12 BW25113,BW25113_4706,0,0.707818


--- Prediction save complete: 0.0 sec ---

=== 't5' classifiers test complete: 30.6 sec ===

=== All test complete: 213.1 sec ===
